In [101]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRFRegressor

In [102]:

df=pd.read_csv('train.csv')

df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [104]:
df=df.drop(['Alley', 'MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1)

In [105]:

simple=SimpleImputer(strategy='most_frequent')
one=OneHotEncoder(sparse_output=False)

cat_col=df.select_dtypes(include='object').columns
num_col=df.select_dtypes(include=['int64','float64']).columns

df[cat_col]=simple.fit_transform(df[cat_col])
catx=one.fit_transform(df[cat_col])


Cat_data=pd.DataFrame(catx,columns=one.get_feature_names_out())

new_df=pd.concat([Cat_data,df[num_col]],axis=1)



In [106]:
X=new_df.drop('SalePrice',axis=1)
y=new_df['SalePrice']

y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [107]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.8,random_state=42)

In [108]:
num_col=X.select_dtypes(include=['int64','float64']).columns

In [109]:
num_pipe=Pipeline(
    steps=[('simple',SimpleImputer(strategy='mean')),
           ('scalar',StandardScaler())])

In [110]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num',num_pipe,num_col)
    ]
)

In [111]:
model=Pipeline(
    steps=[
        ('pre',preprocessing),
        ('randaom',RandomForestRegressor(random_state=42))
    ]
)

In [112]:
model.fit(Xtrain,ytrain)

,steps,"[('pre', ...), ('randaom', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [113]:
model.score(Xtest,ytest)

0.8929665890273053

In [114]:
model.score(Xtrain,ytrain)

0.9790785998320544

In [115]:
df1=pd.read_csv('test.csv')

In [134]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   float64
 1   MSSubClass     1459 non-null   float64
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   float64
 5   Street         1459 non-null   object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1459 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   float64
 17  OverallCond    1459 non-null   float64
 18  YearBuil

In [136]:
df1=df1.drop(['MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1)

KeyError: "['MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'] not found in axis"

In [117]:
cat_col=df1.select_dtypes(include='object').columns
num_col=df1.select_dtypes(exclude='object').columns

In [ ]:
simple=SimpleImputer(strategy='most_frequent')
one=OneHotEncoder(sparse_output=False)


df1[cat_col]=simple.fit_transform(df1[cat_col])
catx=one.fit_transform(df1[cat_col])
catx


Cat_data=pd.DataFrame(catx,columns=one.get_feature_names_out())



array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], shape=(1459, 214))

In [123]:
simple_num=SimpleImputer(strategy='mean')
scalar=StandardScaler()
df1[num_col]=simple_num.fit_transform(df1[num_col])

new_df=pd.concat([Cat_data,df1[num_col]],axis=1)
df1[num_col]


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461.0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,...,730.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0
1,1462.0,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,...,312.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0
2,1463.0,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,...,482.0,212.0,34.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
3,1464.0,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,...,470.0,360.0,36.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0
4,1465.0,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,...,506.0,0.0,82.0,0.0,0.0,144.0,0.0,0.0,1.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915.0,160.0,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0
1455,2916.0,160.0,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,...,286.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0
1456,2917.0,20.0,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,...,576.0,474.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0
1457,2918.0,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,...,0.0,80.0,32.0,0.0,0.0,0.0,0.0,700.0,7.0,2006.0


In [120]:
df1['SalePrice']=model.predict(new_df)

ValueError: columns are missing: {'Utilities_NoSeWa', 'Condition2_RRAn', 'RoofMatl_Metal', 'RoofMatl_ClyTile', 'Exterior1st_ImStucc', 'Heating_OthW', 'RoofMatl_Membran', 'HouseStyle_2.5Fin', 'RoofMatl_Roll', 'Condition2_RRNn', 'Exterior1st_Stone', 'Electrical_Mix', 'Exterior2nd_Other', 'Condition2_RRAe', 'Heating_Floor', 'GarageQual_Ex'}

In [ ]:
result=df1[['Id','SalePrice']]
result

,Id,SalePrice
0,1.0,209410.00
1,2.0,173962.12
2,3.0,222576.35
3,4.0,155617.50
4,5.0,264556.21
...,...,...
1455,1456.0,176008.90
1456,1457.0,206063.80
1457,1458.0,260859.66
1458,1459.0,139209.50


In [ ]:
# cat_pipe=Pipeline(
#     steps=[
#         ('impute',SimpleImputer(strategy='most_frequent')),
#         ('encode',OneHotEncoder(sparse_output=False))
#     ]
# )


# num_pipe=Pipeline(
#     steps=[
#         ('num_impute',SimpleImputer(strategy='mean')),
#         ('scalar',StandardScaler())
#     ]

# )

In [ ]:
# preprocessing=ColumnTransformer(
#     transformers=[
#         ('num',num_pipe,num_col),
#         ('cat',cat_pipe,cat_col)
#     ]
# )

In [139]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# 1. Load data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 2. Separate features and target
X_train = train_df.drop(['Id', 'SalePrice'], axis=1)
y_train = train_df['SalePrice']
X_test = test_df.drop(['Id'], axis=1)

# 3. Fill missing values
for col in X_train.columns:
    if X_train[col].dtype == "object":
        mode_val = X_train[col].mode()[0]
        X_train[col] = X_train[col].fillna(mode_val)
        X_test[col] = X_test[col].fillna(mode_val)
    else:
        mean_val = X_train[col].mean()
        X_train[col] = X_train[col].fillna(mean_val)
        X_test[col] = X_test[col].fillna(mean_val)

# 4. One-hot encode both datasets
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# 5. Align test columns to match training columns
X_test_aligned = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# 6. Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train_encoded, y_train)

# 7. Predict
predictions = model.predict(X_test_aligned)

# 8. Save submission
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': predictions
})
submission.to_csv('submission.csv', index=False)
predictions


array([129101.58, 158066.  , 184863.3 , ..., 155688.5 , 115039.25,
       225487.45], shape=(1459,))

In [138]:
X_test_aligned = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
